# Database Creation

### Importing the COVID Cases to database

In [8]:
# import liberaries
import pandas as pd
import sqlite3

# read in the data
df = pd.read_csv("../data/COVID19Cases_geoRegion_w.csv")

# Select relevant columns to keep the database organized
df_clean = df[[
    "datum", "entries", "sumTotal", "pop", "inz_entries", "inzsumTotal"
]].copy()

# Rename columns for better readability
df_clean.columns = [
    "week", "new_cases", "total_cases", "population",
    "incidence_weekly", "incidence_total"
]


# Create database and write data to the database table
conn = sqlite3.connect("../data/covid_switzerland.db")
df_clean.to_sql("covid_weekly_data", conn, if_exists="replace", index=False)
conn.close()

# Print the first 5 rows of the database to check
conn = sqlite3.connect("../data/covid_switzerland.db")
query = "SELECT * FROM covid_weekly_data LIMIT 5"
df_check = pd.read_sql_query(query, conn)
print(df_check)
conn.close()

     week  new_cases  total_cases  population  incidence_weekly  \
0  202009         57           57     8738791              0.65   
1  202010        377          434     8738791              4.31   
2  202011       2265         2699     8738791             25.92   
3  202012       6558         9257     8738791             75.04   
4  202013       7345        16602     8738791             84.05   

   incidence_total  
0             0.65  
1             4.97  
2            30.89  
3           105.93  
4           189.98  


### Importing the Vaccines to database

In [9]:
# Load the vaccinated persons data
df_vaccinated = pd.read_csv("../data/COVID19VaccPersons_vaccine.csv")

# Select relevant columns (adjust based on the actual column names in the CSV)
df_vaccinated_clean = df_vaccinated[[
    "date", "geoRegion", "vaccine", "entries","type_variant", "pop", "sumTotal" 
]].copy()

# Rename columns for better readability
df_vaccinated_clean.columns = [
    "date", "canton", "vaccine_name", "enteries", "variant_type", "population", "total_vaccinated"
]


# Display the filtered DataFrame
print(df_vaccinated_clean.head())

# Reconnect to the database
conn = sqlite3.connect("../data/covid_switzerland.db")

# Write the vaccinated persons data to a new table
df_vaccinated_clean.to_sql("vaccinated_data", conn, if_exists="replace", index=False)

# Close the connection
conn.close()

# Check the tables in the database
conn = sqlite3.connect("../data/covid_switzerland.db")
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query, conn)
print(tables)
conn.close()

         date canton     vaccine_name  enteries variant_type  population  \
0  22/12/2020     AG  johnson_johnson         0      vaccine    703086.0   
1  23/12/2020     AG  johnson_johnson         0      vaccine    703086.0   
2  24/12/2020     AG  johnson_johnson         0      vaccine    703086.0   
3  25/12/2020     AG  johnson_johnson         0      vaccine    703086.0   
4  26/12/2020     AG  johnson_johnson         0      vaccine    703086.0   

   total_vaccinated  
0                 0  
1                 0  
2                 0  
3                 0  
4                 0  
                name
0  daily_deaths_data
1  covid_weekly_data
2    vaccinated_data


### Importing the Deaths to database

In [10]:
# Load the daily deaths data
df_daily_deaths = pd.read_csv("../data/bag_daily_deaths.csv")

# Display the cleaned DataFrame
print(df_daily_deaths.head())

# Reconnect to the database
conn = sqlite3.connect("../data/covid_switzerland.db")

# Write the daily deaths data to a new table
df_daily_deaths.to_sql("daily_deaths_data", conn, if_exists="replace", index=False)

# Close the connection
conn.close()

# Check the tables in the database
conn = sqlite3.connect("../data/covid_switzerland.db")
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query, conn)
print(tables)
conn.close()

         date  total_deaths  daily_deaths
0  2020-02-24             0             0
1  2020-02-25             0             0
2  2020-02-26             0             0
3  2020-02-27             0             0
4  2020-02-28             0             0
                name
0  covid_weekly_data
1    vaccinated_data
2  daily_deaths_data


### Total Weeks

In [11]:
# Reconnect to the database
conn = sqlite3.connect("../data/covid_switzerland.db")

# Execute the query to count total weeks
query = "SELECT COUNT(*) AS total_weeks FROM covid_weekly_data;"
result = pd.read_sql_query(query, conn)

# Print the result
print(result)

# Close the connection
conn.close()

   total_weeks
0         7056


### Week with the most new cases

In [12]:
# Reconnect to the database
conn = sqlite3.connect("../data/covid_switzerland.db")

# Execute the query to get the week with the highest number of new cases
query = """
SELECT week, new_cases
FROM covid_weekly_data
ORDER BY new_cases DESC
LIMIT 1;
"""
result = pd.read_sql_query(query, conn)

# Print the result
print(result)

# Close the connection
conn.close()

     week  new_cases
0  202204     249378


### Week where the number of cases is over 500

In [13]:
# Reconnect to the database
conn = sqlite3.connect("../data/covid_switzerland.db")

# Execute the query to filter rows with incidence_weekly > 500
query = """
SELECT week, new_cases, incidence_weekly
FROM covid_weekly_data
WHERE incidence_weekly > 500
ORDER BY week;
"""
result = pd.read_sql_query(query, conn)

# Print the result
print(result)

# Close the connection
conn.close()

       week  new_cases  incidence_weekly
0    202042        377            510.85
1    202042       1941            549.53
2    202043         90            550.12
3    202043        207            530.02
4    202043       2665            808.04
..      ...        ...               ...
839  202241       2939            834.51
840  202241       2635            746.02
841  202241       2939            834.51
842  202242       2329            661.31
843  202242       2329            661.31

[844 rows x 3 columns]
